# Select Best ESCO Match for Each PAD Occupation

Send each JSON chunk from ESCO matching to OpenAI API for selection of the best match.

## 0. Setup

### 0.01 Import Required Libraries

In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

from openai import OpenAI

# Import our config
import sys
sys.path.append(str(Path.cwd().parent))
from src.config import load_config

### 0.02 Load Configuration and Environment Variables

In [2]:
# Load environment variables from .env file
project_root = Path.cwd().parent
env_path = project_root / ".env"

if not env_path.exists():
    raise FileNotFoundError(
        f"'.env' file not found at {env_path}\n"
        "Please copy .env.example to .env and add your OpenAI API key."
    )

# Load from specific path
load_dotenv(env_path, override=True)

# Load project config
config = load_config()

# Get OpenAI API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Verify API key is set
if not OPENAI_API_KEY:
    raise ValueError("Missing required environment variable: OPENAI_API_KEY")

print("✓ Environment variables loaded")
print(f"  API Key: {OPENAI_API_KEY[:10]}...{OPENAI_API_KEY[-4:]}")

✓ Environment variables loaded
  API Key: sk-proj-cj...__0A


### 0.03 Set Up Paths

In [3]:
# Set project ID
project_id = "P075941"

# Get paths
input_dir = project_root / "data" / "silver" / "esco_matching_json"
output_dir = project_root / "data" / "silver" / "choose_esco_json"

# Create output directory
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Project ID: {project_id}")
print(f"Input directory: {input_dir}")
print(f"Output directory: {output_dir}")
print(f"Input directory exists: {input_dir.exists()}")

Project ID: P075941
Input directory: /Users/lauren/repos/PAD2Skills/data/silver/esco_matching_json
Output directory: /Users/lauren/repos/PAD2Skills/data/silver/choose_esco_json
Input directory exists: True


## 1. Load JSON Chunk Files

### 1.01 Find JSON Chunk Files

In [4]:
# Find all JSON chunk files for this project
json_files = sorted(input_dir.glob(f"{project_id}_*_esco_matches.json"))

print(f"Found {len(json_files)} JSON chunk files for project {project_id}")
print("\nFiles:")
for json_file in json_files:
    size_kb = json_file.stat().st_size / 1024
    print(f"  {json_file.name:60s} {size_kb:6.2f} KB")

Found 3 JSON chunk files for project P075941

Files:
  P075941_000-074_esco_matches.json                            427.43 KB
  P075941_075-149_esco_matches.json                            423.36 KB
  P075941_150-189_esco_matches.json                            240.61 KB


### 1.02 Inspect Sample Chunk

In [5]:
# Load and inspect first chunk
if json_files:
    with open(json_files[0], 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
    
    print(f"Sample chunk: {json_files[0].name}")
    print(f"Number of records: {len(sample_data)}")
    print(f"\nFirst record:")
    print("=" * 80)
    print(json.dumps(sample_data[0], indent=2))
else:
    print("No JSON files found")

Sample chunk: P075941_000-074_esco_matches.json
Number of records: 75

First record:
{
  "record_id": 0,
  "pad_occupation": "Transmission Line Construction Engineer",
  "pad_activity": "Build power system interconnections to form regional transmission backbone",
  "pad_quote": "The Project will also build important power system interconnections, which will help form the transmission backbone linking Burundi, Rwanda, and Tanzania.",
  "esco_candidates": [
    {
      "rank": 1,
      "esco_id": "7cdeb653-8f3d-4921-832b-b95f9d700a86",
      "label": "electrical transmission system operator",
      "description": "Electrical transmission system operators transport energy in the form of electrical power. They transmit electrical power from generation plants over an interconnected network, an electrical grid, to electricity distribution stations.",
      "similarity_score": 0.82
    },
    {
      "rank": 2,
      "esco_id": "58db3ac6-5217-4d46-8a4c-126598be1d13",
      "label": "electric 

## 2. Process Chunks with OpenAI API

### 2.01 Initialize OpenAI Client

In [6]:
# Initialize OpenAI client
client = OpenAI()

print("✓ OpenAI client initialized")

✓ OpenAI client initialized


### 2.02 Process Each Chunk

In [7]:
print(f"Processing {len(json_files)} JSON chunks...")
print()

processed_chunks = []

for i, json_file in enumerate(json_files, 1):
    # Parse filename to extract chunk identifier: P075941_000-074_esco_matches.json
    # Extract the chunk part (e.g., "000-074")
    filename_stem = json_file.stem  # P075941_000-074_esco_matches
    parts = filename_stem.split('_')
    chunk_id = parts[1]  # 000-074
    
    # Read JSON chunk content
    with open(json_file, 'r', encoding='utf-8') as f:
        chunk_data = json.load(f)
    
    # Convert to JSON string for input message
    input_message = json.dumps(chunk_data, indent=2, ensure_ascii=False)
    
    print(f"[{i}/{len(json_files)}] Processing: {json_file.name}")
    print(f"  Chunk ID: {chunk_id}")
    print(f"  Records: {len(chunk_data)}")
    print(f"  Input size: {len(input_message):,} chars")
    
    # Call OpenAI API with prompt
    response = client.responses.create(
        prompt={
            "id": "pmpt_69570f3e44488197ae85998b411c848b035ce9f8e4648a29",
            "version": "1"
        },
        input=[
            {"role": "user", "content": input_message}
        ],
        reasoning={
            "summary": None
        },
        store=False,
        include=[
            "reasoning.encrypted_content",
            "web_search_call.action.sources"
        ]
    )
    
    # Extract the text from the response
    result = None
    for item in response.output:
        if hasattr(item, 'content') and hasattr(item, 'role'):
            result = item.content[0].text
            break
    
    # Save result to file
    output_file = output_dir / f"{project_id}_{chunk_id}_esco_selection.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(result)
    
    processed_chunks.append({
        'chunk_file': json_file.name,
        'chunk_id': chunk_id,
        'records_in': len(chunk_data),
        'output_file': output_file.name,
        'response_id': response.id
    })
    
    print(f"  ✓ Saved to: {output_file.name}")
    print()

print("=" * 80)
print(f"✓ Processed {len(processed_chunks)} chunks")
print(f"✓ Results saved to: {output_dir}")

Processing 3 JSON chunks...

[1/3] Processing: P075941_000-074_esco_matches.json
  Chunk ID: 000-074
  Records: 75
  Input size: 437,666 chars
  ✓ Saved to: P075941_000-074_esco_selection.json

[2/3] Processing: P075941_075-149_esco_matches.json
  Chunk ID: 075-149
  Records: 75
  Input size: 433,498 chars
  ✓ Saved to: P075941_075-149_esco_selection.json

[3/3] Processing: P075941_150-189_esco_matches.json
  Chunk ID: 150-189
  Records: 40
  Input size: 246,362 chars
  ✓ Saved to: P075941_150-189_esco_selection.json

✓ Processed 3 chunks
✓ Results saved to: /Users/lauren/repos/PAD2Skills/data/silver/choose_esco_json


### 2.03 Verify Output Files

In [8]:
# List all output files
output_files = sorted(output_dir.glob(f"{project_id}_*_esco_selection.json"))

print(f"Created {len(output_files)} output files:")
print("=" * 80)

for output_file in output_files:
    size_kb = output_file.stat().st_size / 1024
    print(f"  {output_file.name:60s} {size_kb:6.2f} KB")

print("=" * 80)
print(f"Total output files: {len(output_files)}")

Created 3 output files:
  P075941_000-074_esco_selection.json                           18.27 KB
  P075941_075-149_esco_selection.json                           18.53 KB
  P075941_150-189_esco_selection.json                            9.76 KB
Total output files: 3


## 3. Combine JSON Files into CSV

### 3.01 Load and Combine JSON Files

In [17]:
import pandas as pd

# Load original PAD data from occupation_skills_csv
pad_csv_path = project_root / "data" / "silver" / "occupation_skills_csv" / f"{project_id}_pad_occupations_prepared.csv"
pad_df = pd.read_csv(pad_csv_path)

print(f"✓ Loaded original PAD data: {len(pad_df)} rows")
print(f"  Columns: {list(pad_df.columns)}")

# Load all JSON selection files
json_selection_files = sorted(output_dir.glob(f"{project_id}_*_esco_selection.json"))

print(f"\nLoading {len(json_selection_files)} JSON selection files...")

all_records = []

for json_file in json_selection_files:
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Extract records from the "results" key
    if isinstance(data, dict) and 'results' in data:
        records = data['results']
        all_records.extend(records)
        print(f"  Loaded: {json_file.name} ({len(records)} records from 'results' key)")
    elif isinstance(data, list):
        all_records.extend(data)
        print(f"  Loaded: {json_file.name} ({len(data)} records - direct list)")
    else:
        all_records.append(data)
        print(f"  Loaded: {json_file.name} (1 record - single dict)")

print(f"\n✓ Combined {len(all_records)} records from {len(json_selection_files)} files")
print(f"\nSample record:")
print(json.dumps(all_records[0], indent=2)[:500])
print("...")

✓ Loaded original PAD data: 190 rows
  Columns: ['extraction_id', 'identified_occupation', 'activity_description_in_pad', 'skills_needed_for_activity', 'source_material_quote', 'project_id', 'section_id', 'pad_id', 'combined_text']

Loading 3 JSON selection files...
  Loaded: P075941_000-074_esco_selection.json (75 records from 'results' key)
  Loaded: P075941_075-149_esco_selection.json (75 records from 'results' key)
  Loaded: P075941_150-189_esco_selection.json (40 records from 'results' key)

✓ Combined 190 records from 3 files

Sample record:
{
  "record_id": 0,
  "chosen_esco": {
    "esco_id": "7cdeb653-8f3d-4921-832b-b95f9d700a86",
    "label": "electrical transmission system operator",
    "rank": 1,
    "confidence": 0.75
  },
  "needs_manual_review": false
}
...


### 3.02 Convert to DataFrame and Save to CSV

In [21]:
# Convert records to DataFrame
# Extract the selection data from each record
df_records = []

for record in all_records:
    # Extract chosen_esco nested object
    chosen_esco = record.get('chosen_esco', {})
    
    # Handle case where chosen_esco is None
    if chosen_esco is None:
        chosen_esco = {}
    
    # Create a record with the selection data
    flat_record = {
        'record_id': record.get('record_id', ''),
        'esco_id': chosen_esco.get('esco_id', ''),
        'esco_label': chosen_esco.get('label', ''),
        'rank': chosen_esco.get('rank', ''),
        'confidence': chosen_esco.get('confidence', ''),
        'needs_manual_review': record.get('needs_manual_review', '')
    }
    df_records.append(flat_record)

# Create DataFrame from selection records
selections_df = pd.DataFrame(df_records)

# Ensure record_id is string type and zero-padded to 3 digits
selections_df['record_id'] = selections_df['record_id'].astype(str).str.zfill(3)

print(f"✓ Created selections DataFrame with {len(selections_df)} rows")
print(f"  Columns: {list(selections_df.columns)}")
print(f"  Sample record_id values: {selections_df['record_id'].head(10).tolist()}")

# Join with original PAD data
# Prepare PAD data with renamed columns
pad_df_prepared = pad_df[['pad_id', 'project_id', 'section_id', 'identified_occupation', 
                           'activity_description_in_pad', 'skills_needed_for_activity', 
                           'source_material_quote']].copy()

# Ensure pad_id is string type before renaming
pad_df_prepared['pad_id'] = pad_df_prepared['pad_id'].astype(str).str.zfill(3)

print(f"\n  PAD data sample pad_id values: {pad_df_prepared['pad_id'].head(10).tolist()}")

pad_df_prepared = pad_df_prepared.rename(columns={
    'pad_id': 'record_id',
    'identified_occupation': 'pad_occupation',
    'activity_description_in_pad': 'pad_activity',
    'skills_needed_for_activity': 'pad_skills',
    'source_material_quote': 'pad_quote',
    'section_id': 'pad_section_id'
})

# Join selections with PAD data
df = selections_df.merge(pad_df_prepared, on='record_id', how='left')

# Reorder columns
column_order = [
    'project_id', 'record_id', 'esco_id', 'esco_label', 'rank', 
    'confidence', 'needs_manual_review',
    'pad_occupation', 'pad_activity', 'pad_skills', 'pad_quote', 'pad_section_id'
]
df = df[column_order]

print(f"\n✓ Joined with PAD data: {len(df)} rows")
print(f"  Final columns: {list(df.columns)}")
print(f"  Non-null pad_occupation count: {df['pad_occupation'].notna().sum()}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

✓ Created selections DataFrame with 190 rows
  Columns: ['record_id', 'esco_id', 'esco_label', 'rank', 'confidence', 'needs_manual_review']
  Sample record_id values: ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009']

  PAD data sample pad_id values: ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009']

✓ Joined with PAD data: 190 rows
  Final columns: ['project_id', 'record_id', 'esco_id', 'esco_label', 'rank', 'confidence', 'needs_manual_review', 'pad_occupation', 'pad_activity', 'pad_skills', 'pad_quote', 'pad_section_id']
  Non-null pad_occupation count: 190

DataFrame shape: (190, 12)

First few rows:


,project_id,record_id,esco_id,esco_label,rank,confidence,needs_manual_review,pad_occupation,pad_activity,pad_skills,pad_quote,pad_section_id
0,P075941,000,7cdeb653-8f3d-4921-832b-b95f9d700a86,electrical transmission system operator,1,0.75,False,Transmission Line Construction Engineer,Build power system interconnections to form re...,"['build power system interconnections', 'form ...",The Project will also build important power sy...,0
1,P075941,001,7052fd94-f563-46a9-8e2d-cba6c20f3e71,overhead line worker,6,0.85,False,Electrical Transmission Line Installer,Build transmission interconnections and form t...,"['build power system interconnections', 'form ...",The Project will also build important power sy...,0
2,P075941,002,58db3ac6-5217-4d46-8a4c-126598be1d13,electric power generation engineer,1,0.9,False,Hydroelectric Power Plant Construction Engineer,Increase regional power generation capacity to...,"['increase regional generation', 'help meet re...",It will play a critical role in increasing reg...,0
3,P075941,003,fcf9024e-277c-43c7-8ec8-8903e5c9c576,water engineer,1,0.8,False,Water Resources Development Specialist,Develop and manage river resources in an integ...,['develop the resources of the Nyabarongo Rive...,This collaboration will ensure that the resour...,0
4,P075941,004,4b214899-ed00-4156-b269-a23df0fb1093,environmental protection manager,1,0.9,False,Environmental Management Specialist,Ensure integrated and environmentally sustaina...,['ensure development is integrated and environ...,This collaboration will ensure that the resour...,0


### 3.03 Add Section Names

In [24]:
import re

# Load document sections JSON
sections_path = project_root / "data" / "silver" / "document_sections" / f"{project_id}_sections.json"

with open(sections_path, 'r', encoding='utf-8') as f:
    sections_data = json.load(f)

# Create mapping from section_id to cleaned header_text
section_mapping = {}
for section in sections_data['sections']:
    header = section['header_text']
    # Remove pound signs
    cleaned = header.replace('#', '')
    # Replace multiple whitespaces with single space
    cleaned = re.sub(r'\s+', ' ', cleaned)
    # Strip leading/trailing whitespace
    cleaned = cleaned.strip()
    section_mapping[section['section_id']] = cleaned

print(f"✓ Loaded {len(section_mapping)} section names")
print(f"\nSample sections:")
for section_id in list(section_mapping.keys()):
    print(f"  {section_id}: {section_mapping[section_id]}")

# Add section name column to dataframe
df['pad_section_name'] = df['pad_section_id'].map(section_mapping)

# Update column order to include section name
column_order = [
    'project_id', 'record_id', 'esco_id', 'esco_label', 'rank', 
    'confidence', 'needs_manual_review',
    'pad_occupation', 'pad_activity', 'pad_skills', 'pad_quote', 
    'pad_section_id', 'pad_section_name'
]
df = df[column_order]

print(f"\n✓ Added section names to dataframe")
print(f"  Non-null pad_section_name count: {df['pad_section_name'].notna().sum()}")
print(f"\nUpdated DataFrame:")
df.head()

✓ Loaded 16 section names

Sample sections:
  0: I. STRATEGIC CONTEXT
  1: II. PROJECT DEVELOPMENT OBJECTIVES
  2: III. PROJECT DESCRIPTION
  3: IV. IMPLEMENTATION
  4: V. KEY RISKS AND MITIGATION MEASURES
  5: VI. APPRAISAL SUMMARY
  6: Annex 1: Results Framework and Monitoring
  7: Annex 2: Detailed Project Description
  8: Annex 3: Implementation Arrangements Regional Rusumo Falls Hydroelectric Project
  9: Annex 4: Operational Risk Assessment Framework (ORAF)
  10: Annex 5: Economic and Financial Analysis Implementation Arrangements
  11: Annex 6: Power Supply Options for the Nile Equatorial Lakes Region (NEL)
  12: Annex 7: Summary of the Power Sectors in Burundi, Rwanda and Tanzania
  13: Annex 8: Implementation Support Team
  14: Annex 9: Communication Strategy
  15: Annex 10: Documents in the Project file

✓ Added section names to dataframe
  Non-null pad_section_name count: 190

Updated DataFrame:


,project_id,record_id,esco_id,esco_label,rank,confidence,needs_manual_review,pad_occupation,pad_activity,pad_skills,pad_quote,pad_section_id,pad_section_name
0,P075941,000,7cdeb653-8f3d-4921-832b-b95f9d700a86,electrical transmission system operator,1,0.75,False,Transmission Line Construction Engineer,Build power system interconnections to form re...,"['build power system interconnections', 'form ...",The Project will also build important power sy...,0,I. STRATEGIC CONTEXT
1,P075941,001,7052fd94-f563-46a9-8e2d-cba6c20f3e71,overhead line worker,6,0.85,False,Electrical Transmission Line Installer,Build transmission interconnections and form t...,"['build power system interconnections', 'form ...",The Project will also build important power sy...,0,I. STRATEGIC CONTEXT
2,P075941,002,58db3ac6-5217-4d46-8a4c-126598be1d13,electric power generation engineer,1,0.9,False,Hydroelectric Power Plant Construction Engineer,Increase regional power generation capacity to...,"['increase regional generation', 'help meet re...",It will play a critical role in increasing reg...,0,I. STRATEGIC CONTEXT
3,P075941,003,fcf9024e-277c-43c7-8ec8-8903e5c9c576,water engineer,1,0.8,False,Water Resources Development Specialist,Develop and manage river resources in an integ...,['develop the resources of the Nyabarongo Rive...,This collaboration will ensure that the resour...,0,I. STRATEGIC CONTEXT
4,P075941,004,4b214899-ed00-4156-b269-a23df0fb1093,environmental protection manager,1,0.9,False,Environmental Management Specialist,Ensure integrated and environmentally sustaina...,['ensure development is integrated and environ...,This collaboration will ensure that the resour...,0,I. STRATEGIC CONTEXT


### 3.04 Save CSV

In [25]:
# Create output directory for CSV
csv_output_dir = project_root / "data" / "silver" / "choose_esco_csv"
csv_output_dir.mkdir(parents=True, exist_ok=True)

# Save to CSV
csv_output_file = csv_output_dir / f"{project_id}_esco_selections.csv"
df.to_csv(csv_output_file, index=False)

print(f"✓ Saved combined selections to: {csv_output_file}")
print(f"  File size: {csv_output_file.stat().st_size / 1024:.2f} KB")
print(f"  Rows: {len(df):,}")
print(f"  Columns: {len(df.columns)}")
print(f"\nCSV contains:")
print(f"  - PAD occupation details (record_id, occupation, activity, quote)")
print(f"  - Selected ESCO match (ID, label, description)")
print(f"  - Selection reasoning")

✓ Saved combined selections to: /Users/lauren/repos/PAD2Skills/data/silver/choose_esco_csv/P075941_esco_selections.csv
  File size: 123.03 KB
  Rows: 190
  Columns: 13

CSV contains:
  - PAD occupation details (record_id, occupation, activity, quote)
  - Selected ESCO match (ID, label, description)
  - Selection reasoning


## 4. Create Unique ESCO Matches File

### 4.01 Load and Prepare Data

In [26]:
# Read the esco_selections CSV
selections_csv_path = project_root / "data" / "silver" / "choose_esco_csv" / f"{project_id}_esco_selections.csv"
df_selections = pd.read_csv(selections_csv_path)

print(f"✓ Loaded selections data: {len(df_selections)} rows")
print(f"  Columns: {list(df_selections.columns)}")

# Drop records where needs_manual_review = True
df_filtered = df_selections[df_selections['needs_manual_review'] != True].copy()
dropped_count = len(df_selections) - len(df_filtered)

print(f"\n✓ Filtered out records with needs_manual_review=True")
print(f"  Dropped: {dropped_count} rows")
print(f"  Remaining: {len(df_filtered)} rows")

# Add pad_section_name to the front of each pad_quote
df_filtered['pad_quote'] = df_filtered['pad_section_name'] + ': "' + df_filtered['pad_quote'] + '"'

print(f"\n✓ Formatted pad_quote with section names")
print(f"\nSample formatted quote:")
print(f"  {df_filtered['pad_quote'].iloc[0][:200]}...")

print(f"\nPrepared DataFrame:")
df_filtered.head()

✓ Loaded selections data: 190 rows
  Columns: ['project_id', 'record_id', 'esco_id', 'esco_label', 'rank', 'confidence', 'needs_manual_review', 'pad_occupation', 'pad_activity', 'pad_skills', 'pad_quote', 'pad_section_id', 'pad_section_name']

✓ Filtered out records with needs_manual_review=True
  Dropped: 3 rows
  Remaining: 187 rows

✓ Formatted pad_quote with section names

Sample formatted quote:
  I. STRATEGIC CONTEXT: "The Project will also build important power system interconnections, which will help form the transmission backbone linking Burundi, Rwanda, and Tanzania."...

Prepared DataFrame:


,project_id,record_id,esco_id,esco_label,rank,confidence,needs_manual_review,pad_occupation,pad_activity,pad_skills,pad_quote,pad_section_id,pad_section_name
0,P075941,0,7cdeb653-8f3d-4921-832b-b95f9d700a86,electrical transmission system operator,1.0,0.75,False,Transmission Line Construction Engineer,Build power system interconnections to form re...,"['build power system interconnections', 'form ...","I. STRATEGIC CONTEXT: ""The Project will also b...",0,I. STRATEGIC CONTEXT
1,P075941,1,7052fd94-f563-46a9-8e2d-cba6c20f3e71,overhead line worker,6.0,0.85,False,Electrical Transmission Line Installer,Build transmission interconnections and form t...,"['build power system interconnections', 'form ...","I. STRATEGIC CONTEXT: ""The Project will also b...",0,I. STRATEGIC CONTEXT
2,P075941,2,58db3ac6-5217-4d46-8a4c-126598be1d13,electric power generation engineer,1.0,0.90,False,Hydroelectric Power Plant Construction Engineer,Increase regional power generation capacity to...,"['increase regional generation', 'help meet re...","I. STRATEGIC CONTEXT: ""It will play a critical...",0,I. STRATEGIC CONTEXT
3,P075941,3,fcf9024e-277c-43c7-8ec8-8903e5c9c576,water engineer,1.0,0.80,False,Water Resources Development Specialist,Develop and manage river resources in an integ...,['develop the resources of the Nyabarongo Rive...,"I. STRATEGIC CONTEXT: ""This collaboration will...",0,I. STRATEGIC CONTEXT
4,P075941,4,4b214899-ed00-4156-b269-a23df0fb1093,environmental protection manager,1.0,0.90,False,Environmental Management Specialist,Ensure integrated and environmentally sustaina...,['ensure development is integrated and environ...,"I. STRATEGIC CONTEXT: ""This collaboration will...",0,I. STRATEGIC CONTEXT


### 4.02 Flatten Data by ESCO ID

In [28]:
import ast

# Function to format skills (remove brackets and add quotes)
def format_skills(skills_str):
    if pd.isna(skills_str):
        return ''
    try:
        # Parse the string as a list
        skills_list = ast.literal_eval(skills_str)
        # Add quotes around each skill and join with comma
        return ', '.join([f'"{skill}"' for skill in skills_list])
    except:
        # If parsing fails, return as is
        return skills_str

# Group by esco_id and aggregate
grouped = df_filtered.groupby(['project_id', 'esco_id', 'esco_label']).agg({
    'pad_occupation': lambda x: ', '.join([f'"{occupation}"' for occupation in x.dropna().astype(str).unique()]),
    'pad_activity': lambda x: ', '.join([f'"{activity}"' for activity in x.dropna().astype(str).unique()]),
    'pad_skills': lambda x: ', '.join([format_skills(skill) for skill in x.dropna().astype(str).unique()]),
    'pad_quote': lambda x: ', '.join(x.dropna().astype(str).unique())
}).reset_index()

# Rename columns
grouped = grouped.rename(columns={
    'pad_occupation': 'pad_occupations',
    'pad_activity': 'pad_activities',
    'pad_quote': 'pad_quotes'
})

print(f"✓ Flattened data by esco_id")
print(f"  Unique ESCO IDs: {len(grouped)}")
print(f"  Columns: {list(grouped.columns)}")
print(f"\nDataFrame shape: {grouped.shape}")
print(f"\nFirst few rows:")
grouped.head()

✓ Flattened data by esco_id
  Unique ESCO IDs: 91
  Columns: ['project_id', 'esco_id', 'esco_label', 'pad_occupations', 'pad_activities', 'pad_skills', 'pad_quotes']

DataFrame shape: (91, 7)

First few rows:


,project_id,esco_id,esco_label,pad_occupations,pad_activities,pad_skills,pad_quotes
0,P075941,00030d09-2b3a-4efd-87cc-c4ea39d27c34,technical director,"""Technical Manager""","""Manage all technical matters relating to the ...","""manage technical matters relating to the proj...",Annex 4: Operational Risk Assessment Framework...
1,P075941,03b2f112-a891-461d-a8d4-effb9dcf2df6,quarry engineer,"""Rock Excavation Engineer (Rockfill Materials)""","""Excavate material required for rockfill at th...","""excavate material required for rockfill at th...","Annex 2: Detailed Project Description: ""The ma..."
2,P075941,0561328b-875b-4ae2-9ba1-9af9049aef01,procurement category specialist,"""Procurement Specialist""","""member of the core implementation-support tea...","""provide implementation support on procurement...","Annex 8: Implementation Support Team: ""The cor..."
3,P075941,0ba06640-e0ac-4911-9e43-289a8e41651e,corporate trainer,"""Financial Trainer (Livelihood Restoration Pro...","""Deliver financial training as part of the liv...","""financial training for sustainable utilizatio...","VI. APPRAISAL SUMMARY: ""In addition all affect..."
4,P075941,0cca32c2-9308-4927-adb2-14771ab787f0,quantity surveyor,"""Quantity Surveyor""","""Provide quantity surveying services to suppor...","""support the set-up operations of the Rusumo P...","III. PROJECT DESCRIPTION: ""This component will..."


### 4.03 Add ESCO URI and Merge with ESCO Data

In [29]:
# Create esco_uri from esco_id
grouped['esco_uri'] = 'http://data.europa.eu/esco/occupation/' + grouped['esco_id']

print(f"✓ Created esco_uri field")
print(f"  Sample URI: {grouped['esco_uri'].iloc[0]}")

# Load ESCO occupations data
esco_path = project_root / "data" / "bronze" / "esco" / "occupations_en.csv"
esco_df = pd.read_csv(esco_path)

print(f"\n✓ Loaded ESCO occupations data: {len(esco_df)} rows")
print(f"  Columns: {list(esco_df.columns)}")

# Select relevant columns from ESCO data
esco_subset = esco_df[['conceptUri', 'description', 'naceCode']].copy()
esco_subset = esco_subset.rename(columns={
    'conceptUri': 'esco_uri',
    'description': 'esco_description',
    'naceCode': 'nace_code'
})

# Merge with grouped data
df_unique = grouped.merge(esco_subset, on='esco_uri', how='left')

# Reorder columns
column_order = [
    'project_id', 'esco_id', 'esco_label', 'esco_description', 'nace_code',
    'pad_occupations', 'pad_activities', 'pad_skills', 'pad_quotes', 'esco_uri'
]
df_unique = df_unique[column_order]

print(f"\n✓ Merged with ESCO data")
print(f"  Final rows: {len(df_unique)}")
print(f"  Columns: {list(df_unique.columns)}")
print(f"  Non-null esco_description count: {df_unique['esco_description'].notna().sum()}")
print(f"\nFinal DataFrame:")
df_unique.head()

✓ Created esco_uri field
  Sample URI: http://data.europa.eu/esco/occupation/00030d09-2b3a-4efd-87cc-c4ea39d27c34

✓ Loaded ESCO occupations data: 3043 rows
  Columns: ['conceptType', 'conceptUri', 'iscoGroup', 'preferredLabel', 'altLabels', 'hiddenLabels', 'status', 'modifiedDate', 'regulatedProfessionNote', 'scopeNote', 'definition', 'inScheme', 'description', 'code', 'naceCode']

✓ Merged with ESCO data
  Final rows: 91
  Columns: ['project_id', 'esco_id', 'esco_label', 'esco_description', 'nace_code', 'pad_occupations', 'pad_activities', 'pad_skills', 'pad_quotes', 'esco_uri']
  Non-null esco_description count: 91

Final DataFrame:


,project_id,esco_id,esco_label,esco_description,nace_code,pad_occupations,pad_activities,pad_skills,pad_quotes,esco_uri
0,P075941,00030d09-2b3a-4efd-87cc-c4ea39d27c34,technical director,Technical directors realise the artistic visio...,http://data.europa.eu/ux2/nace2.1/9031,"""Technical Manager""","""Manage all technical matters relating to the ...","""manage technical matters relating to the proj...",Annex 4: Operational Risk Assessment Framework...,http://data.europa.eu/esco/occupation/00030d09...
1,P075941,03b2f112-a891-461d-a8d4-effb9dcf2df6,quarry engineer,Quarry engineers analyse which extraction meth...,"http://data.europa.eu/ux2/nace2.1/7499,\nhttp:...","""Rock Excavation Engineer (Rockfill Materials)""","""Excavate material required for rockfill at th...","""excavate material required for rockfill at th...","Annex 2: Detailed Project Description: ""The ma...",http://data.europa.eu/esco/occupation/03b2f112...
2,P075941,0561328b-875b-4ae2-9ba1-9af9049aef01,procurement category specialist,Procurement category specialists are experts i...,http://data.europa.eu/ux2/nace2.1/702,"""Procurement Specialist""","""member of the core implementation-support tea...","""provide implementation support on procurement...","Annex 8: Implementation Support Team: ""The cor...",http://data.europa.eu/esco/occupation/0561328b...
3,P075941,0ba06640-e0ac-4911-9e43-289a8e41651e,corporate trainer,"Corporate trainers train, coach, and guide emp...",http://data.europa.eu/ux2/nace2.1/8559,"""Financial Trainer (Livelihood Restoration Pro...","""Deliver financial training as part of the liv...","""financial training for sustainable utilizatio...","VI. APPRAISAL SUMMARY: ""In addition all affect...",http://data.europa.eu/esco/occupation/0ba06640...
4,P075941,0cca32c2-9308-4927-adb2-14771ab787f0,quantity surveyor,Quantity surveyors have under their helm compl...,"http://data.europa.eu/ux2/nace2.1/7112,\nhttp:...","""Quantity Surveyor""","""Provide quantity surveying services to suppor...","""support the set-up operations of the Rusumo P...","III. PROJECT DESCRIPTION: ""This component will...",http://data.europa.eu/esco/occupation/0cca32c2...


### 4.04 Save Unique ESCO Matches

In [30]:
# Create output directory
unique_output_dir = project_root / "data" / "silver" / "unique_esco_csv"
unique_output_dir.mkdir(parents=True, exist_ok=True)

# Save to CSV
unique_output_file = unique_output_dir / f"{project_id}_unique_matched.csv"
df_unique.to_csv(unique_output_file, index=False)

print(f"✓ Saved unique ESCO matches to: {unique_output_file}")
print(f"  File size: {unique_output_file.stat().st_size / 1024:.2f} KB")
print(f"  Rows: {len(df_unique):,}")
print(f"  Columns: {len(df_unique.columns)}")
print(f"\nCSV contains:")
print(f"  - Unique ESCO occupations with aggregated PAD data")
print(f"  - ESCO descriptions and NACE codes")
print(f"  - Collapsed occupations, activities, skills, and quotes")

✓ Saved unique ESCO matches to: /Users/lauren/repos/PAD2Skills/data/silver/unique_esco_csv/P075941_unique_matched.csv
  File size: 156.73 KB
  Rows: 91
  Columns: 10

CSV contains:
  - Unique ESCO occupations with aggregated PAD data
  - ESCO descriptions and NACE codes
  - Collapsed occupations, activities, skills, and quotes
